# Chapter 17 classification examples

This notebook uses some existing tools, other than those used by the textbook, to implement the examples seen in Chapter 17.

In [251]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
path_data = 'http://personal.psu.edu/drh20/200DS/assets/data/'

## 17.1. Nearest Neighbors

[Section 17.1](https://inferentialthinking.com/chapters/17/1/Nearest_Neighbors.html) uses the `ckd` dataset on patients with or without chronic kidney disease to build a nearest-neighbor classifier.  We'll use the `sklearn` implementation of nearest neighbors.

In [252]:
from sklearn import neighbors

Read in the CKD dataset, then create the predictor dataframe and response array:

In [ ]:
ckd = pd.read_csv(path_data + 'ckd.csv')
ckd.columns

In [ ]:
ckd_pred = ckd.iloc[:, [14, 9]] # Grab the Hemoglobin and Glucose columns
ckd_pred = ckd_pred.rename(columns = {'Blood Glucose Random': 'Glucose'})
ckd_pred = (ckd_pred-ckd_pred.mean())/ckd_pred.std() # Standardize each column
ckd_resp = np.array(ckd.iloc[:, 24]) # Grab the Class column
ckd_pred # Take a look at the predictor DataFrame

Now a scatterplot of Hemoglobin vs. Glucose:

In [ ]:
plt.figure(figsize=(8, 6))

for resp, color, lab in zip([0, 1], ['blue', 'orange'], ['No CKD', 'CKD']):
  x = ckd_pred.loc[ckd_resp==resp, 'Hemoglobin']
  y = ckd_pred.loc[ckd_resp==resp, 'Glucose']
  plt.scatter(x=x, y=y, c=color, label=lab)
  
plt.legend()
plt.grid(True)
plt.show()

Next, define a new patient named Alice and give her (standardized versions of) hemoglobin and glucose:

In [ ]:
alice = pd.DataFrame(data={'Hemoglobin': [0.0], 'Glucose': [1.5]})

plt.figure(figsize=(8, 6))

for resp, color, lab in zip([0, 1], ['blue', 'orange'], ['No CKD', 'CKD']):
  x = ckd_pred.loc[ckd_resp==resp, 'Hemoglobin']
  y = ckd_pred.loc[ckd_resp==resp, 'Glucose']
  plt.scatter(x=x, y=y, c=color, label=lab)  

plt.scatter(x=alice.loc[0,'Hemoglobin'], y=alice.loc[0,'Glucose'], c='red', label='Alice')

plt.legend()
plt.grid(True)
plt.show()

Now we can train a k-nearest neighbors classifier using k=1, then use it to classify Alice.  From the look of the plot, Alice's nearest neighbor is a CKD individual, which means that she should be classifed as 1:

In [ ]:
knn1 = neighbors.KNeighborsClassifier(n_neighbors=1)
knn1.fit(ckd_pred, ckd_resp)
knn1.predict(alice)

We can check that for a 1-nearest neighbor classifier, we get a 100% "success" rate if we apply it to the training dataset:

In [ ]:
knn1.predict(ckd_pred) == ckd_resp

If instead we train a 5-nearest neighbors classifier, we no longer get 100% correct in the training set:

In [ ]:
knn5 = neighbors.KNeighborsClassifier(n_neighbors=5)
knn5.fit(ckd_pred, ckd_resp)
knn5.predict(ckd_pred) == ckd_resp

Finally, we can create a mesh of grid points, then classify each one, and this will give us a decision boundary:

In [ ]:
h = 0.02  # step size in the mesh

x_min, x_max = ckd_pred.iloc[:, 0].min() - 1, ckd_pred.iloc[:, 0].max() + 1
y_min, y_max = ckd_pred.iloc[:, 1].min() - 1, ckd_pred.iloc[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
mesh = pd.DataFrame().assign(Hemoglobin=xx.ravel(), Glucose=yy.ravel())
Z = knn1.predict(mesh)

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, alpha=0.3)

for resp, color, lab in zip([0, 1], ['blue', 'orange'], ['No CKD', 'CKD']):
  x = ckd_pred.loc[ckd_resp==resp, 'Hemoglobin']
  y = ckd_pred.loc[ckd_resp==resp, 'Glucose']
  plt.scatter(x=x, y=y, c=color, label=lab, edgecolor='black')
  
plt.legend()
plt.title('1-Nearest Neighbor Classifier')
plt.xlabel('Hemoglobin standard units')
plt.ylabel('Glucose standard units')
plt.show()

## 17.2. Training and Testing

[Section 17.2](https://inferentialthinking.com/chapters/17/2/Training_and_Testing.html) also uses the CKD dataset, but with different columns chosen as predictors as compared with [Section 17.1](https://inferentialthinking.com/chapters/17/1/Nearest_Neighbors.html). 

In [261]:
ckd_pred = ckd.iloc[:, [16, 9]] # Grab the White Blood Cell Count and Glucose columns
ckd_pred = ckd_pred.rename(columns = {'Blood Glucose Random': 'Glucose'})
ckd_pred = (ckd_pred-ckd_pred.mean())/ckd_pred.std() # Standardize each column
ckd_resp = np.array(ckd.iloc[:, 24]) # Grab the Class column

Here is a scatterplot of the two variables:

In [ ]:
plt.figure(figsize=(8, 6))

for resp, color, lab in zip([0, 1], ['blue', 'orange'], ['No CKD', 'CKD']):
  x = ckd_pred.loc[ckd_resp==resp, 'White Blood Cell Count']
  y = ckd_pred.loc[ckd_resp==resp, 'Glucose']
  plt.scatter(x=x, y=y, c=color, label=lab)
  
plt.legend()
plt.grid(True)
plt.xlabel('White Blood Cell Count')
plt.ylabel('Glucose')
plt.show()

We can randomly split the dataset into a training set and an equal-sized test set:

In [263]:
shuffle = np.random.choice(158, 158, replace=False)
train = shuffle[0:79] # this is the first half of a randomly reshuffled array from 0 to 157
test = shuffle[79:158] # this is the second half
train_pred = ckd_pred.iloc[train, :]
train_resp = ckd_resp[train]
test_pred = ckd_pred.iloc[test,:]
test_resp = ckd_resp[test]

Let's now train a 1-nearest neighbor classifier using the training set and check how it does on the test set:

In [ ]:
knn1 = neighbors.KNeighborsClassifier(n_neighbors=1)
knn1.fit(train_pred, train_resp)
knn1.predict(test_pred) == test_resp

Finally, we can depict the decision boundary and the test set on the same plot:

In [ ]:
h = 0.02  # step size in the mesh

x_min, x_max = train_pred.iloc[:, 0].min() - 1, train_pred.iloc[:, 0].max() + 1
y_min, y_max = train_pred.iloc[:, 1].min() - 1, train_pred.iloc[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
mesh = pd.DataFrame().assign(a=xx.ravel(), Glucose=yy.ravel()).rename(columns = {'a': 'White Blood Cell Count'})
Z = knn1.predict(mesh) # This makes sure that the background contour plot is based on training data

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, alpha=0.3)

# Now add test points
for resp, color, lab in zip([0, 1], ['blue', 'orange'], ['No CKD', 'CKD']):
  x = test_pred.loc[test_resp==resp, 'White Blood Cell Count']
  y = test_pred.loc[test_resp==resp, 'Glucose']
  plt.scatter(x=x, y=y, c=color, label=lab, edgecolor='black')
  
plt.legend()
plt.title('1-Nearest Neighbor Classifier')
plt.xlabel('White Blood Cell Count standard units')
plt.ylabel('Glucose standard units')
plt.show()

## 17.3. Rows of Tables

Much of [Section 17.3](https://inferentialthinking.com/chapters/17/3/Rows_of_Tables.html) uses the capabilities of `numpy`, rather than the `datascience` library.  Thus, we can mostly repeat what was done in that section without much change.  

In [266]:
ckd_pred = ckd.iloc[:, [14, 9]] # Grab the Hemoglobin and Glucose columns
ckd_pred = ckd_pred.rename(columns = {'Blood Glucose Random': 'Glucose'})
ckd_pred = (ckd_pred-ckd_pred.mean())/ckd_pred.std() # Standardize each column
ckd_resp = np.array(ckd.iloc[:, 24]) # Grab the Class column

Here is a new point called `Alice` and a distance function used to calculate the distance between Alice and a particular row of the `ckd_pred` dataset:

In [ ]:
alice = np.array([0, 1.1])
def distance(point1, point2): #Returns the Euclidean distance between point1 and point2.
    return np.sqrt(np.sum((point1 - point2)**2))
distance(alice, ckd_pred.iloc[46,:])

Now we can create a new function called `distance_from_alice` just like in [Section 17.3](https://inferentialthinking.com/chapters/17/3/Rows_of_Tables.html), then apply this function to each row in `ckd_pred`.  The main difference between this code and the code in the textbook is that the `apply` function here works on a pandas `DataFrame` object, and this means that we have to tell it which set of indices (in this case, the column indices or `axis=1`) to use:

In [ ]:
def distance_from_alice(row): #Returns distance between Alice and a row of predictors
    return distance(alice, np.array(row))
distances = ckd_pred.apply(distance_from_alice, axis=1)

# Now display a DataFrame in which the distance column is added, then used to sort:
ckd_pred.assign(distance_from_alice=distances).sort_values('distance_from_alice')

Finally, we can produce the final plot from [Section 17.3](https://inferentialthinking.com/chapters/17/3/Rows_of_Tables.html) without changing much of the original code from the textbook.  This plot shows Alice surrounded by a circle of radius equal to the fifth-smallest distance:

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(ckd_pred['Hemoglobin'], ckd_pred['Glucose'], c=ckd_resp, s=40)
plt.scatter(alice[0], alice[1], c='red', s=40)
radius = np.sort(distances)[4]
theta = np.arange(0, 2*np.pi+1, 2*np.pi/200)
plt.plot(radius*np.cos(theta)+alice[0], radius*np.sin(theta)+alice[1], c='g', lw=1.5);
plt.xlim(-2, 2.5)
plt.ylim(-2, 2.5)
plt.grid(True);

## 17.4. Implementing the Classifier

[Section 17.4](https://inferentialthinking.com/chapters/17/4/Implementing_the_Classifier.html) begins with a new dataset with measurements on banknotes, some of which are counterfeit and some of which are not.  Here is code that reads in the dataset and then produces a scatterplot like the first one in Section 17.4:

In [ ]:
banknotes = pd.read_csv(path_data + 'banknote.csv')
notes_resp = np.array(banknotes['Class'])

plt.figure(figsize=(8, 6))

for resp, color, lab in zip([0, 1], ['blue', 'orange'], ['Genuine', 'Counterfeit']):
  x = banknotes.loc[notes_resp==resp, 'WaveletVar']
  y = banknotes.loc[notes_resp==resp, 'WaveletCurt']
  plt.scatter(x=x, y=y, c=color, label=lab)
  
plt.legend()
plt.grid(True)
plt.xlabel('WaveletVar')
plt.ylabel('WaveletCurt')
plt.show()

And here is a 3-d plot like the one seen in [Section 17.4](https://inferentialthinking.com/chapters/17/4/Implementing_the_Classifier.html):

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = Axes3D(fig)

for grp_name, grp_idx in banknotes.groupby('Class').groups.items():
    x = banknotes.loc[grp_idx,'WaveletVar']
    y = banknotes.loc[grp_idx,'WaveletSkew']
    z = banknotes.loc[grp_idx,'WaveletCurt']
    ax.scatter(x,y,z, label=grp_name)

ax.legend(labels=['Genuine', 'Counterfeit']);

Finally, here is an interactive version of the same 3-d plot:

In [ ]:
import plotly.express as px
df = banknotes.assign(size=2.0)
fig = px.scatter_3d(df, x='WaveletVar', y='WaveletSkew', z='WaveletCurt',
              color='Class', size = 'size', opacity=0.5)
fig.show()

Next, [Section 17.4](https://inferentialthinking.com/chapters/17/4/Implementing_the_Classifier.html) considers the `wine` dataset.  This code reads the dataset and splits it into predictors and response:

In [272]:
wines = pd.read_csv(path_data + 'wine.csv')
wine_pred = wines.iloc[:,1:14] # columns 2 through 14 are predictors
wine_resp = wines.iloc[:,0] # column 1 is the response (category of wine)

Here is an illustration that we can use the same `distance` function defined in [Section 17.3](https://inferentialthinking.com/chapters/17/3/Rows_of_Tables.html) on 14-dimensional predictors like in this example:

In [ ]:
distance(wine_pred.iloc[0,:], wine_pred.iloc[1,:])

In Section 17.4.5, several functions are defined.  We can create versions of each of them.  The `distance` function has already been defined above.

In [274]:
def all_distances(training, new_point): # Return array of dist between each training point and new_point
    def distance_from_new_point(row):
        return distance(np.array(new_point), np.array(row))
    return training.apply(distance_from_new_point, axis=1)

def table_with_distances(training, new_point): # Return DataFrame with all_distances appended
    return training.assign(Distance=all_distances(training, new_point))

def closest(training, new_point, k): # Return array of indices of k closest rows 
    return np.array(table_with_distances(training, new_point).sort_values('Distance').index[0:k])

To duplicate what happens in [Section 17.4](https://inferentialthinking.com/chapters/17/4/Implementing_the_Classifier.html), let's take row 0 to be the `new_point` and find the indices of the 5 closest rows to that point.  (Which row index will show up as the closest row?)

In [ ]:
five_closest = closest(wine_pred, wine_pred.iloc[0,:], 5)
five_closest

By referring to the original dataset and looking at the 5 closest rows, we can see that the response values of those 5 nearest rows are all the same, which means that a 5-nearest neighbors classifier would consider row 0 to have `Class` equal to 1:

In [ ]:
wines.iloc[five_closest,:]

Remember, we don't have to implement the k-nearest neighbors classifier ourselves like the textbook does.  We can use the `sklearn` library's code for this purpose instead.

## 17.5. The accuracy of the classifier

[Section 17.5](https://inferentialthinking.com/chapters/17/5/Accuracy_of_the_Classifier.html) begins by splitting the `wines` dataset into two equal-sized datasets, one for training and one for testing:

In [287]:
shuffle = np.random.choice(178, 178, replace=False)
train = shuffle[0:89] # this is the first half of a randomly reshuffled array from 0 to 177
test = shuffle[89:178] # this is the second half
train_pred = wine_pred.iloc[train, :]
train_resp = wine_resp[train]
test_pred = wine_pred.iloc[test,:]
test_resp = wine_resp[test]

In [ ]:
np.sum(test_resp) + np.sum(train_resp) - np.sum(wine_resp)

Now we can train the 5-nearest neighbor classifier and test it:

In [ ]:
knn5 = neighbors.KNeighborsClassifier(n_neighbors=5)
knn5.fit(train_pred, train_resp)
np.array(knn5.predict(test_pred) == test_resp)

Taking the mean of True/False is a fast way to see the proportion of True:

In [ ]:
np.mean(knn5.predict(test_pred) == test_resp)

Our accuracy is slightly lower than the accuracy seen in the textbook because the textbook reduced the number of wine categories from 3 to 2.

## 17.6. Multiple Regression
[Section 17.6](https://inferentialthinking.com/chapters/17/6/Multiple_Regression.html) uses the `minimize` function to minimize the RMSE (root mean squared error) as a function of the 9-dimensional array of slope values.  However, the minimizer can be found directly using multivariable calculus, which is the way that all regression software actually works.  You don't need to understand the calculus for this, but a bit of matrix notation knowledge helps:

The slopes that minimize the mean square error are given by
$$ (X^\top X)^{-1} X^\top Y, $$
where $X$ is the matrix of predictors, $Y$ is the vector (array) of response values, $X^\top$ is the transpose of $X$, and $(X^\top X)^{-1}$ is the matrix inverse of $X^\top X$.

Let's read in the house price dataset and set up the predictor matrix and response array:

In [207]:
house = pd.read_csv(path_data + 'house.csv')

# First, filter so we only get the rows (houses) we want:
house = house.loc[house['Bldg Type']=='1Fam', :] # only 1Fam houses
house = house.loc[house['Sale Condition']=='Normal', :] # only Normal sale condition

# Second, select only the subset of columns (variables) we want:
house = house[['SalePrice', '1st Flr SF', '2nd Flr SF', 
    'Total Bsmt SF', 'Garage Area', 
    'Wood Deck SF', 'Open Porch SF', 'Lot Area', 
    'Year Built', 'Yr Sold']]

# Third, sort by sale price (to match Section 17.6):
house = house.sort_values('SalePrice')

# Finally, create house_pred and house_resp arrays:
house_pred = np.array(house.iloc[:, 1:10]) # Convert the DataFrame to an array here!
house_resp = np.array(house.iloc[:, 0])

Split the datasets into training and test of equal size:

In [229]:
shuffle = np.random.choice(2002, 2002, replace=False)
train = shuffle[0:1001] # this is the first half of a randomly reshuffled array from 0 to 177
test = shuffle[1001:2002] # this is the second half
train_pred = house_pred[train, :] # don't need '.iloc' here because it's an array, not a DataFrame
train_resp = house_resp[train]
test_pred = house_pred[test,:] # don't need '.iloc' here because it's an array, not a DataFrame
test_resp = house_resp[test]

Now we can use the formula above to find the best slopes:  $(X^\top X)^{-1} X^\top Y$.  Compare with the values in Section 17.6.2.1, keeping in mind that they will be slightly different because the training subset is randomly selected:

In [ ]:
X = train_pred
Y = train_resp
XtXinv = np.linalg.inv(np.matmul(np.transpose(X), X))
XtY = np.matmul(np.transpose(X), Y)
best_slopes = np.matmul(XtXinv, XtY)
np.round(best_slopes)

To predict a sales price based on an array of predictor values, we can use the same `predict` function from Section 17.6.2:

In [ ]:
def predict(slopes, row):
    return sum(slopes * np.array(row))

row = 0
test_row = test_pred[row,:]
print('Actual price ', test_resp[row])
print('Predicted price ', predict(best_slopes, test_pred[row, :]) )